In [1]:
!pip install pytorch-crf

In [3]:
import pandas as pd
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
from torch import nn
from torchcrf import CRF

In [4]:
def load_conll_for_bilstm(file_path, max_len=None, lowercase=True):
    """
    Converts CoNLL data into indexed tensors for BiLSTM NER.
    
    Args:
        file_path: path to .conll or .txt file
        max_len: optional max sequence length for padding
        lowercase: whether to convert tokens to lowercase
    
    Returns:
        sentences_padded: Tensor (num_sentences, max_len)
        tags_padded: Tensor (num_sentences, max_len)
        token2idx, label2idx: dictionaries for mapping
    """
    sentences, labels = [], []
    sent, tag_seq = [], []

    # Read and parse CoNLL file
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:  # end of sentence
                if sent:
                    sentences.append(sent)
                    labels.append(tag_seq)
                    sent, tag_seq = [], []
            else:
                parts = line.split()
                token = parts[0]   # first column = value
                if lowercase:
                    token = token.lower()  # convert to lowercase
                label = parts[-1]  # last column = label
                sent.append(token)
                tag_seq.append(label)
        # Add last sentence if not empty
        if sent:
            sentences.append(sent)
            labels.append(tag_seq)

    # Build vocabularies
    all_tokens = {t for s in sentences for t in s}
    all_labels = {l for lab in labels for l in lab}

    token2idx = {t: i + 2 for i, t in enumerate(sorted(all_tokens))}
    token2idx["<PAD>"] = 0
    token2idx["<UNK>"] = 1

    label2idx = {l: i for i, l in enumerate(sorted(all_labels))}
    pad_label_id = len(label2idx)
    label2idx["<PAD>"] = pad_label_id

    # Convert to index sequences
    X = [[token2idx.get(t, 1) for t in s] for s in sentences]
    y = [[label2idx[l] for l in lab] for lab in labels]

    # Convert to PyTorch tensors
    X_tensors = [torch.tensor(seq, dtype=torch.long) for seq in X]
    y_tensors = [torch.tensor(seq, dtype=torch.long) for seq in y]

    # Pad sequences
    if max_len is None:
        max_len = max(len(seq) for seq in X_tensors)

    sentences_padded = pad_sequence(
        X_tensors, batch_first=True, padding_value=token2idx["<PAD>"]
    )
    tags_padded = pad_sequence(
        y_tensors, batch_first=True, padding_value=label2idx["<PAD>"]
    )

    # Truncate if specified
    sentences_padded = sentences_padded[:, :max_len]
    tags_padded = tags_padded[:, :max_len]

    return sentences_padded, tags_padded, token2idx, label2idx


In [ ]:
# from sklearn.model_selection import train_test_split

# train_path = "/kaggle/input/annotation/train.condll"

# train_X, train_y, token2idx, label2idx = load_conll_for_bilstm(train_path)

# print("Full dataset:", train_X.shape, train_y.shape)

# train_X, val_X, train_y, val_y = train_test_split(
#     train_X, train_y, test_size=0.2, random_state=42
# )

# print("Train split :", train_X.shape, train_y.shape)
# print("Val split   :", val_X.shape, val_y.shape)

Full dataset: torch.Size([137, 30]) torch.Size([137, 30])
Train split : torch.Size([109, 30]) torch.Size([109, 30])
Val split   : torch.Size([28, 30]) torch.Size([28, 30])


In [6]:
train_path = "/kaggle/input/annotate/train.conll"
test_path = "/kaggle/input/annotate/test.conll"

train_X, train_y, token2idx, label2idx = load_conll_for_bilstm(train_path)
test_X, test_y, _, _ = load_conll_for_bilstm(test_path)

print(train_X.shape) 
print(train_y.shape)

torch.Size([346, 31])
torch.Size([346, 31])


In [7]:
import torch
import torch.nn as nn
from torchcrf import CRF

class MaskedAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Linear(hidden_dim, 1)

    def forward(self, lstm_out, mask):
        # lstm_out: (batch, seq_len, hidden_dim)
        scores = self.attn(lstm_out).squeeze(-1)
        scores = scores.masked_fill(mask == 0, -1e9)
        attn_weights = torch.softmax(scores, dim=1).unsqueeze(-1)
        context = torch.sum(lstm_out * attn_weights, dim=1, keepdim=True)
        return lstm_out + context.expand_as(lstm_out)


class BiLSTM_CRF(nn.Module):
    def __init__(self, vocab_size, tagset_size, embedding_dim=128, hidden_dim=256, num_layers=2, dropout=0.25, pad_idx=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.embedding_dropout = nn.Dropout(dropout)

        self.bilstm = nn.LSTM(
            embedding_dim,
            hidden_dim // 2,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )

        self.layer_norm = nn.LayerNorm(hidden_dim)
        self.attention = MaskedAttention(hidden_dim)

        # optional hidden projection before CRF
        self.hidden_fc = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

        self.fc = nn.Linear(hidden_dim, tagset_size)
        self.crf = CRF(tagset_size, batch_first=True)

    def forward(self, x, tags=None, mask=None):
        if mask is None:
            mask = (x != self.embedding.padding_idx).type(torch.bool)
        mask[:, 0] = 1

        embeddings = self.embedding(x)
        embeddings = self.embedding_dropout(embeddings)

        lstm_out, _ = self.bilstm(embeddings)
        lstm_out = self.layer_norm(lstm_out)
        lstm_out = self.attention(lstm_out, mask)
        lstm_out = self.hidden_fc(lstm_out)

        emissions = self.fc(lstm_out)

        if tags is not None:
            log_likelihood = self.crf(emissions, tags, mask=mask)
            return -log_likelihood.mean()
        else:
            return self.crf.decode(emissions, mask=mask)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

X = train_X.to(device)
y = train_y.to(device)

# Use boolean mask
mask = (X != token2idx["<PAD>"]).to(device).bool()

model = BiLSTM_CRF(
    vocab_size=len(token2idx),
    tagset_size=len(label2idx),
    embedding_dim=128,
    hidden_dim=256,
    pad_idx=token2idx["<PAD>"]
).to(device)

Using device: cuda


In [9]:
from torch import optim

optimizer = optim.Adam(model.parameters(), lr=1e-2)
num_epochs = 50

model.train()
for epoch in range(num_epochs):
    optimizer.zero_grad()
    loss = model(X, y, mask=mask)   # forward pass
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
    optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

Epoch 1/50, Loss: 15378.2373
Epoch 2/50, Loss: 21755.3750
Epoch 3/50, Loss: 15868.3740
Epoch 4/50, Loss: 14093.6465
Epoch 5/50, Loss: 16939.2910
Epoch 6/50, Loss: 9070.6289
Epoch 7/50, Loss: 7212.6260
Epoch 8/50, Loss: 7308.2671
Epoch 9/50, Loss: 6196.1685
Epoch 10/50, Loss: 4910.9473
Epoch 11/50, Loss: 5052.1987
Epoch 12/50, Loss: 4611.6992
Epoch 13/50, Loss: 3630.7588
Epoch 14/50, Loss: 3402.2129
Epoch 15/50, Loss: 3060.9097
Epoch 16/50, Loss: 2540.0642
Epoch 17/50, Loss: 2406.5054
Epoch 18/50, Loss: 2061.3201
Epoch 19/50, Loss: 1802.6008
Epoch 20/50, Loss: 1610.7126
Epoch 21/50, Loss: 1502.1685
Epoch 22/50, Loss: 1211.6895
Epoch 23/50, Loss: 1113.4557
Epoch 24/50, Loss: 1012.1248
Epoch 25/50, Loss: 850.7432
Epoch 26/50, Loss: 844.5101
Epoch 27/50, Loss: 703.1438
Epoch 28/50, Loss: 624.6495
Epoch 29/50, Loss: 586.9553
Epoch 30/50, Loss: 538.3320
Epoch 31/50, Loss: 578.0394
Epoch 32/50, Loss: 547.5119
Epoch 33/50, Loss: 471.8807
Epoch 34/50, Loss: 405.0645
Epoch 35/50, Loss: 401.9098


In [10]:
idx2label = {idx: label for label, idx in label2idx.items()}

In [11]:
# Eval mode
model.eval()
with torch.no_grad():
    predictions = model(X, mask=mask)   # list[list[int]]

print("predictions type:", type(predictions))
print("first pred length:", len(predictions[0]), "expected length:", int(mask[0].sum().item()))

# Robust flattening using mask sums
mask_bool = (X != token2idx["<PAD>"])  
y_true_flat = y[mask_bool].cpu().numpy()

y_pred_flat = []
for seq_pred, seq_mask in zip(predictions, mask_bool):
    seq_len = int(seq_mask.sum().item())
    y_pred_flat.extend(seq_pred[:seq_len])

y_pred_flat = np.array(y_pred_flat)

# Exclude PAD labels if present
pad_id = label2idx.get("<PAD>", None)
if pad_id is not None:
    valid = (y_true_flat != pad_id)
    y_true_final = y_true_flat[valid]
    y_pred_final = y_pred_flat[valid]
else:
    y_true_final = y_true_flat
    y_pred_final = y_pred_flat

from sklearn.metrics import f1_score, accuracy_score

f1 = f1_score(y_true_final, y_pred_final, average='micro')
acc = accuracy_score(y_true_final, y_pred_final)
print("f1:", f1, "accuracy:", acc)

predictions type: <class 'list'>
first pred length: 3 expected length: 3
f1: 0.9742639890816923 accuracy: 0.9742639890816923


In [ ]:
idx2label = {v:k for k,v in label2idx.items()}

model.eval()
with torch.no_grad():
    preds = model(X, mask=mask)

for i in range(min(3, X.size(0))):
    seq_len = int(mask[i].sum().item())
    true_seq = y[i, :seq_len].cpu().tolist()
    pred_seq = preds[i][:seq_len]
    print("TRUE:", [idx2label[t] for t in true_seq])
    print("PRED:", [idx2label[p] for p in pred_seq])
    print()

TRUE: ['O', 'B-DEP', 'I-DEP']
PRED: ['O', 'B-DEP', 'I-DEP']

TRUE: ['O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC']
PRED: ['O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC']

TRUE: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
PRED: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']



In [13]:
import re

sentence = '''
CHÍNH PH Ủ 
--------  CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM  
Độc lập - Tự do - Hạnh phúc  
---------------  
Số: 83/NQ -CP Hà N ội, ngày  08 tháng  6 năm 2024  
  
NGHỊ QUYẾT 
VỀ DỰ ÁN NGH Ị QUY ẾT CỦA QU ỐC HỘI VỀ GIẢM THU Ế GIÁ TR Ị GIA TĂNG  
CHÍNH PH Ủ 
Căn c ứ Luật Tổ chức Chính ph ủ ngày 19 tháng 6 năm 2015; Lu ật sửa đổi, bổ sung m ột 
số điều của Luật Tổ chức Chính ph ủ và Lu ật Tổ chức Chính quy ền địa phương ngày 22 
tháng 11 năm 2019;  
Căn c ứ Luật ban hành văn b ản quy ph ạm pháp lu ật ngày 22 tháng 6 năm 2015; Lu ật 
sửa đổi, bổ sung m ột số điều của Luật ban hành văn b ản quy ph ạm pháp lu ật ngày 18 
tháng 6 năm 2020;  
Căn c ứ Nghị định số 39/2022/NĐ -CP ngày 18 tháng 6 năm 2022 c ủa Chính ph ủ ban 
hành Quy ch ế làm vi ệc của Chính ph ủ; 
Xét đ ề nghị của Bộ trưởng Bộ Tài chính t ại Tờ trình s ố 127/TTr -BTC ngày 06 tháng 6 
năm 2024;  
Trên cơ s ở kết quả biểu quy ết của các Thành viên Chính ph ủ, 
QUYẾT NGHỊ: 
Điều 1.

'''

lower_sentence = sentence.lower().strip()

# Tokenize the data to match the Label-Studio style of splitting tokens
def tokenize_like_conll(text):
    """
    Split tokens like Label Studio:
    - Separate punctuation (/,:; etc.)
    - Split numbers, letters, symbols
    """
    return re.findall(r"\w+|[^\w\s]", text, re.UNICODE)

tokens = tokenize_like_conll(lower_sentence)
print("Tokens:", tokens)

# Convert tokens to indices 
X_test = [token2idx.get(tok, token2idx["<UNK>"]) for tok in tokens]

# Convert to tensor
X_test_tensor = torch.tensor([X_test], dtype=torch.long).to(device)  # shape: (1, seq_len)

# Create attention mask
mask_test = (X_test_tensor != token2idx["<PAD>"]).to(torch.bool)

print("Tensor shape:", X_test_tensor.shape)
print("Mask shape:", mask_test.shape)

Tokens: ['chính', 'ph', 'ủ', '-', '-', '-', '-', '-', '-', '-', '-', 'cộng', 'hòa', 'xã', 'hội', 'chủ', 'nghĩa', 'việt', 'nam', 'độc', 'lập', '-', 'tự', 'do', '-', 'hạnh', 'phúc', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'số', ':', '83', '/', 'nq', '-', 'cp', 'hà', 'n', 'ội', ',', 'ngày', '08', 'tháng', '6', 'năm', '2024', 'nghị', 'quyết', 'về', 'dự', 'án', 'ngh', 'ị', 'quy', 'ết', 'của', 'qu', 'ốc', 'hội', 'về', 'giảm', 'thu', 'ế', 'giá', 'tr', 'ị', 'gia', 'tăng', 'chính', 'ph', 'ủ', 'căn', 'c', 'ứ', 'luật', 'tổ', 'chức', 'chính', 'ph', 'ủ', 'ngày', '19', 'tháng', '6', 'năm', '2015', ';', 'lu', 'ật', 'sửa', 'đổi', ',', 'bổ', 'sung', 'm', 'ột', 'số', 'điều', 'của', 'luật', 'tổ', 'chức', 'chính', 'ph', 'ủ', 'và', 'lu', 'ật', 'tổ', 'chức', 'chính', 'quy', 'ền', 'địa', 'phương', 'ngày', '22', 'tháng', '11', 'năm', '2019', ';', 'căn', 'c', 'ứ', 'luật', 'ban', 'hành', 'văn', 'b', 'ản', 'quy', 'ph', 'ạm', 'pháp', 'lu', 'ật', 'ngày', '22', 'tháng', '6', 'năm'

In [14]:
X_test_tensor = X_test_tensor.to(device)
mask_test = mask_test.to(device)

model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor, mask=mask_test)

pred_labels = [idx2label[idx] for idx in predictions[0]]

In [15]:
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor, mask=mask_test)

pred_labels = [idx2label[idx] for idx in predictions[0]]

In [16]:
for token, label in zip(tokens, pred_labels):
    print(f"{token}: {label}")

chính: B-DEP
ph: I-DEP
ủ: I-DEP
-: O
-: O
-: O
-: O
-: O
-: O
-: O
-: O
cộng: B-LOC
hòa: I-LOC
xã: I-LOC
hội: I-LOC
chủ: I-LOC
nghĩa: I-LOC
việt: I-LOC
nam: I-LOC
độc: O
lập: O
-: O
tự: O
do: O
-: O
hạnh: O
phúc: O
-: O
-: O
-: O
-: O
-: O
-: O
-: O
-: O
-: O
-: O
-: O
-: O
-: O
-: O
-: O
số: O
:: O
83: B-DOCID
/: I-DOCID
nq: I-DOCID
-: O
cp: O
hà: B-LOC
n: I-LOC
ội: I-LOC
,: O
ngày: B-DAT
08: I-DAT
tháng: I-DAT
6: I-DAT
năm: I-DAT
2024: I-DAT
nghị: O
quyết: B-TIT
về: B-TIT
dự: I-TIT
án: I-TIT
ngh: I-TIT
ị: I-TIT
quy: I-TIT
ết: I-TIT
của: I-TIT
qu: I-TIT
ốc: I-TIT
hội: I-TIT
về: I-TIT
giảm: I-TIT
thu: I-TIT
ế: I-TIT
giá: I-TIT
tr: I-TIT
ị: I-TIT
gia: I-TIT
tăng: I-TIT
chính: B-DEP
ph: I-DEP
ủ: I-DEP
căn: O
c: O
ứ: O
luật: B-TIT
tổ: I-TIT
chức: I-TIT
chính: I-TIT
ph: I-TIT
ủ: I-TIT
ngày: B-DAT
19: I-DAT
tháng: I-DAT
6: I-DAT
năm: I-DAT
2015: I-DAT
;: O
lu: B-TIT
ật: I-TIT
sửa: I-TIT
đổi: I-TIT
,: I-TIT
bổ: I-TIT
sung: I-TIT
m: I-TIT
ột: I-TIT
số: I-TIT
điều: I-TIT
của: O
luật: B-TIT
tổ:

In [17]:
### Save model
import torch
import json

# Example: paths to save
MODEL_PATH = "model_bilstm_crf.pt"
TOKEN_PATH = "token2idx.json"
LABEL_PATH = "label2idx.json"

# --- Save model weights ---
torch.save(model.state_dict(), MODEL_PATH)

# --- Save dictionaries ---
with open(TOKEN_PATH, "w", encoding="utf-8") as f:
    json.dump(token2idx, f, ensure_ascii=False, indent=2)

with open(LABEL_PATH, "w", encoding="utf-8") as f:
    json.dump(label2idx, f, ensure_ascii=False, indent=2)

print("✅ Model and dictionaries saved successfully!")

✅ Model and dictionaries saved successfully!
